# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. **Задача** - построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
pip install sweetviz

In [2]:
import pandas as pd
import sweetviz as sv
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.dummy import DummyRegressor

In [3]:
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('/datasets/autos.csv')
display(df.head())
df.info()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
report = sv.analyze(df)
report.show_html()

При анализе выявлено:
- есть 4 дубликата;
- в столбцах 'VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired' есть пропущенные значения;
- в столбцах 'Price', 'RegistrationYear', 'Power' есть аномальные значения;
- в столбце 'NumberOfPictures' только значения 0;
- есть категориальные признаки, которые необходимо перевести в численные.

### Обработка пропущенных значений и дубликатов

In [6]:
# удалим дубликаты и сделаем проверку
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

Разберёмся с пропущенными значениями. В столбце 'VehicleType' пропущенных значений 11%, в 'Model' - 6%, в 'FuelType' - 9%. Достаточно большие части данных, чтобы их удалять. Поэтому заменим пропущенные значения в этих признаках на 'other', так как такое значение у этих признаков уже существует.

In [7]:
df[['VehicleType', 'Model', 'FuelType']] = df[['VehicleType', 'Model', 'FuelType']].fillna('other')
df['VehicleType'].isna().sum()

0

Тип коробки передач 'Gearbox' невозможно определить, пропущенных значений - 6%. Можно поставить заглушку. 

Также типов коробки передач только два - автомат и механика. Можно заменить значения 'manual' на 0, 'auto' на 1, а пропущенные значения на -1. Потом преобразовать значения в целочисленный тип.

In [8]:
df['Gearbox'] = df['Gearbox'].replace('manual', 0)
df['Gearbox'] = df['Gearbox'].replace('auto', 1)
df['Gearbox'] = df['Gearbox'].fillna(-1)
df['Gearbox'] = df['Gearbox'].astype('int')

В столбце 'NotRepaired' пропущенных значений очень много - 20%. Сделаем аналогичные действия, как и со столбцом 'Gearbox'.

In [9]:
df['NotRepaired'] = df['NotRepaired'].replace('no', 0)
df['NotRepaired'] = df['NotRepaired'].replace('yes', 1)
df['NotRepaired'] = df['NotRepaired'].fillna(-1)
df['NotRepaired'] = df['NotRepaired'].astype('int')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354365 entries, 0 to 354364
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354365 non-null  object
 1   Price              354365 non-null  int64 
 2   VehicleType        354365 non-null  object
 3   RegistrationYear   354365 non-null  int64 
 4   Gearbox            354365 non-null  int64 
 5   Power              354365 non-null  int64 
 6   Model              354365 non-null  object
 7   Kilometer          354365 non-null  int64 
 8   RegistrationMonth  354365 non-null  int64 
 9   FuelType           354365 non-null  object
 10  Brand              354365 non-null  object
 11  NotRepaired        354365 non-null  int64 
 12  DateCreated        354365 non-null  object
 13  NumberOfPictures   354365 non-null  int64 
 14  PostalCode         354365 non-null  int64 
 15  LastSeen           354365 non-null  object
dtypes: int64(9), object(

**Вывод:** пропущенных значений нет, категориальных признаков стало меньше.

### Обработка аномальных значений

Аномальные значения встречаются в столбцах 'Price', 'RegistrationYear' и 'Power'. Разберём каждый по порядку.

In [11]:
df['Price'].describe()

count    354365.000000
mean       4416.679830
std        4514.176349
min           0.000000
25%        1050.000000
50%        2700.000000
75%        6400.000000
max       20000.000000
Name: Price, dtype: float64

Цена автомобилей в датафрейме варьируется от 0 до 20000. Максимум вполне реальный, а вот цена в 0 евро быть не может. Также есть значения, которые слишком малы для цены. Изучив сайты продажи авто, приходим к выводу, что цены чаще всего начинаются от 50 евро (например, за авто на запчасти). Посмотрим, какую долю данных строки с ценой меньше 50 евро.

In [12]:
round(len(df.query('Price <50'))/len(df)*100,2)

3.51

3,51% небольшая доля данных, которые можно убрать.

Изучим столбец 'RegistrationYear'.

In [13]:
df['RegistrationYear'].describe()

count    354365.000000
mean       2004.234481
std          90.228466
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: RegistrationYear, dtype: float64

В этом столбце есть также слишком малые и большие значения для года регистрации авто. Возьмём границы для авто с 1920 по нынешний год 2022.

In [14]:
round(len(df.query('1920 <= RegistrationYear <= 2022'))/len(df)*100,2)

99.92

Доля таких строк составляет 99,9%. То есть, убирая строки с аномальными значениями, мы почти не теряем никаких данных.

Изучим столбец 'Power'.

In [15]:
df['Power'].describe()

count    354365.000000
mean        110.093816
std         189.851330
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: Power, dtype: float64

Учитывая, что мощность старых автомобилей может быть около 33 л.с., а самый мощный авто имеет 2000 л.с., то возьмем интервал мощности от 30 до 2000 л.с.

In [16]:
round(len(df.query('30 <= Power <= 2000'))/len(df)*100,2)

88.36

Учтём все условия и ограничения и посмотрим, какая в итоге доля от изначальных данных у нас останется.

In [17]:
df_new = df.query('Price > 50 and 1920 <= RegistrationYear <= 2022 and 30 <= Power <= 2000')
round(len(df_new)/len(df)*100, 2)

86.14

Получилось, что если мы уберём эти строки, то потеряем около 14% данных. Доля немаленькая, но чтобы модель работала корректно, необходимы и корректные данные. Поэтому в последующем будем работать с ДатаФреймом `df_new`.

### Подготовка признаков

Используя дату создания анкеты `DateCreated` и год регистрации авто `RegistrationYear` вычислим возраст автомобиля в годах. После этого `DateCreated` можно будет убрать из признаков, так как возраст авто влияет на его стоимость явно больше, чем дата создания анкеты.

In [19]:
df_new['DateCreated'] = pd.DatetimeIndex(df_new['DateCreated']).year
df_new['Age'] = df_new['DateCreated'] - df_new['RegistrationYear']
df_new.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen,Age
1,2016-03-24 10:58:45,18300,coupe,2011,0,190,other,125000,5,gasoline,audi,1,2016,0,66954,2016-04-07 01:46:50,5
2,2016-03-14 12:52:21,9800,suv,2004,1,163,grand,125000,8,gasoline,jeep,-1,2016,0,90480,2016-04-05 12:47:46,12
3,2016-03-17 16:54:04,1500,small,2001,0,75,golf,150000,6,petrol,volkswagen,0,2016,0,91074,2016-03-17 17:40:17,15
4,2016-03-31 17:25:20,3600,small,2008,0,69,fabia,90000,7,gasoline,skoda,0,2016,0,60437,2016-04-06 10:17:21,8
5,2016-04-04 17:36:23,650,sedan,1995,0,102,3er,150000,10,petrol,bmw,1,2016,0,33775,2016-04-06 19:17:07,21


Выделим целевой признак и остальные. Так как в столбце 'NumberOfPictures' везде значения 0, это столбец неинформативен, поэтому уберём его из признаков. Также не влияют на стоимость авто дата скачивания анкеты и дата последней активности пользователя, их тоже не будем брать в качестве признаков. Почтовый индекс пользователя также маловероятно связан с ценой авто, уберем из признаков и его.

In [20]:
target = df_new['Price']
features = df_new.drop(['Price', 'NumberOfPictures', 'DateCrawled', 'PostalCode', 'LastSeen', 'DateCreated'], axis=1)

Теперь разделим все данные на тестовую и обучающую выборки.

In [21]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

Так как есть категориальные признаки их необходимо преобразовать. Сделаем это с помощью `OrdinalEncoder()`. Обучим энкодер на категориальных признаках тренировочной выборки, а трансформируем признаки и тренировочной, и тестовой выборок.

In [22]:
encoder = OrdinalEncoder()
categorial = ['VehicleType', 'Model', 'FuelType', 'Brand']
encoder.fit(features_train[categorial])
features_train[categorial] = encoder.transform(features_train[categorial])
features_test[categorial] = encoder.transform(features_test[categorial])

In [24]:
features_train.head()

,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,Age
44619,6.0,2006,1,218,117.0,150000,12,2.0,14.0,0,10
232890,2.0,1997,0,75,73.0,150000,6,5.0,11.0,-1,19
47137,5.0,2002,0,86,127.0,150000,12,6.0,34.0,0,14
161893,4.0,2009,0,143,6.0,70000,7,2.0,2.0,0,7
338504,5.0,2013,0,131,83.0,60000,1,2.0,24.0,0,3


### Вывод

В ходе подготовки данных было сделано:
- удаление дубликатов и заполнение пропущенных значений;
- анализ аномальных значений и удаление части данных для их однородности;
- перевод категориальных переменных в численные;
- разделение данных на тестовую и обучающую выборки.

## Обучение моделей

Для обучения моделей воспользуемся `GridSearchCV()`. Обучим несколько моделей: Градиентный бустинг (CatBoost, LightGBM), Дерево решений, Случайный лес. Для Линейной регрессии используем кросс-валидацию.

### CatBoost

In [25]:
parameters_cbr = {'learning_rate' : [0.5, 0.6, 0.7, 0.8, 0.9],
                 'depth' : range (1, 10)
                }

In [26]:
%%time

CBR = GridSearchCV(CatBoostRegressor(iterations=50), parameters_cbr, scoring='neg_root_mean_squared_error')
CBR.fit(features_train, target_train)
rmse = CBR.best_score_
best_model_cbr = CBR.best_estimator_
print(CBR.best_params_)
print('RMSE модели:', abs(rmse.round(2)))

0:	learn: 3996.8288299	total: 70.2ms	remaining: 3.44s
1:	learn: 3648.3007292	total: 87.5ms	remaining: 2.1s
2:	learn: 3453.5932385	total: 105ms	remaining: 1.65s
3:	learn: 3317.5203868	total: 122ms	remaining: 1.41s
4:	learn: 3180.1796830	total: 139ms	remaining: 1.25s
5:	learn: 3091.3407877	total: 156ms	remaining: 1.14s
6:	learn: 3003.5015060	total: 176ms	remaining: 1.08s
7:	learn: 2917.7641512	total: 193ms	remaining: 1.01s
8:	learn: 2858.8646498	total: 212ms	remaining: 964ms
9:	learn: 2813.6151817	total: 232ms	remaining: 930ms
10:	learn: 2762.1528253	total: 251ms	remaining: 890ms
11:	learn: 2724.6140047	total: 270ms	remaining: 855ms
12:	learn: 2691.9439691	total: 289ms	remaining: 822ms
13:	learn: 2664.3883908	total: 310ms	remaining: 797ms
14:	learn: 2635.7047648	total: 330ms	remaining: 771ms
15:	learn: 2607.2195217	total: 349ms	remaining: 741ms
16:	learn: 2584.6564575	total: 370ms	remaining: 718ms
17:	learn: 2563.4295733	total: 390ms	remaining: 694ms
18:	learn: 2544.1160780	total: 411ms	

### LightGBM

In [27]:
parameters_lgbm = {'learning_rate' : [0.7, 0.8, 0.9],
                   'max_depth' : range (10, 15)
                  }

In [28]:
%%time

LGBMR = GridSearchCV(LGBMRegressor(), parameters_lgbm, scoring='neg_root_mean_squared_error')
LGBMR.fit(features_train, target_train)
best_model_lgbmr = LGBMR.best_estimator_
rmse = LGBMR.best_score_
print(LGBMR.best_params_)
print('Значение RMSE:', abs(rmse.round(2)))

{'learning_rate': 0.7, 'max_depth': 10}
Значение RMSE: 1662.55
CPU times: user 45min 35s, sys: 35.8 s, total: 46min 11s
Wall time: 46min 30s


### Дерево решений

In [29]:
state = np.random.RandomState(12345)

In [30]:
parameters_dtr = {'max_depth': range (1,15,2),
                 'min_samples_leaf': range (1,10),
                 'min_samples_split': range(2,10)
                }

In [31]:
%%time

DTR = GridSearchCV(DecisionTreeRegressor(random_state=state), parameters_dtr, scoring='neg_root_mean_squared_error', cv=5)
DTR.fit(features_train, target_train)
best_model_dtr = DTR.best_estimator_
rmse = DTR.best_score_
print(DTR.best_params_)
print('Значение RMSE:', abs(rmse.round(2)))

{'max_depth': 13, 'min_samples_leaf': 8, 'min_samples_split': 2}
Значение RMSE: 1863.82
CPU times: user 14min 54s, sys: 2.73 s, total: 14min 57s
Wall time: 14min 57s


###  Случайный лес

In [32]:
parameters_rfr = {'max_depth': range (10,60,10),
                 'n_estimators': range (10, 60, 20)
                }

In [33]:
%%time
RFR = GridSearchCV(RandomForestRegressor(random_state=state), parameters_rfr, scoring='neg_root_mean_squared_error', cv=5)
RFR.fit(features_train, target_train)
best_model_rfr = RFR.best_estimator_
rmse = RFR.best_score_
print(RFR.best_params_)
print('Значение RMSE:', abs(rmse.round(2)))

{'max_depth': 20, 'n_estimators': 50}
Значение RMSE: 1609.49
CPU times: user 22min 33s, sys: 21.2 s, total: 22min 54s
Wall time: 22min 55s


### Линейная регрессия

In [34]:
%%time

LR = LinearRegression()
scores_LR = cross_val_score(LR, features_train, target_train, scoring='neg_root_mean_squared_error', cv=5)
rmse = abs(scores_LR).mean() # берём оценки по модулю, так как 'neg_root_mean_squared_error' возвращает отрицательные значения
print('Значение RMSE:', rmse.round(2))

Значение RMSE: 3239.44
CPU times: user 577 ms, sys: 372 ms, total: 949 ms
Wall time: 942 ms


- Самой качественной моделью получился Случайный лес с параметрами `max_depth = 20` и `n_estimators = 50`. Значение RMSE на валидационной выборке получилось 1609.49. Но по времени обучения модель занимает 22 минуты, что по сравнению с CatBoost и Деревом решений достаточно долго, поэтому её не рекомендуется выбирать.
- Самой быстрой моделью и по обучению, и по предсказанию получилась Линейная регрессия, но качество модели очень низкое. Значение RMSE 3239.44.
- Учитывая скорость и качество модели, необходимое заказчику, рекомендовано использовать модель Градиентного бустинга CatBoost с параметрами `depth = 9` и `learning_rate = 0.5`. Время обучения у модели 3 секунды, а предсказания 20 миллисекунд, а значение RMSE = 1645.72. 

## Анализ моделей

На данном шаге мы будем анализировать выбранную модель по скорости и качеству, используя тестовую выборку.

### CatBoost

In [35]:
%%time

CBR_test = best_model_cbr
CBR_test.fit(features_train, target_train, verbose=10)

0:	learn: 3145.8669551	total: 57.7ms	remaining: 2.83s
10:	learn: 1819.6270594	total: 597ms	remaining: 2.12s
20:	learn: 1729.7137320	total: 1.11s	remaining: 1.53s
30:	learn: 1675.9589749	total: 1.61s	remaining: 985ms
40:	learn: 1636.6946939	total: 2.15s	remaining: 473ms
49:	learn: 1609.0591993	total: 2.62s	remaining: 0us
CPU times: user 2.81 s, sys: 91.8 ms, total: 2.9 s
Wall time: 3.46 s


In [36]:
%%time

rmse = mean_squared_error(target_test, CBR_test.predict(features_test)) ** 0.5
print('Значение RMSE:', rmse.round(2))

Значение RMSE: 1646.01
CPU times: user 24.2 ms, sys: 39 µs, total: 24.2 ms
Wall time: 22.4 ms


Проверим модель на адекватность, используя Дамми модель.

### Случайная модель

In [37]:
%%time

dummy_regr = DummyRegressor()
dummy_regr.fit(features_train, target_train)
print('RMSE случайной модели:', mean_squared_error(target_test, dummy_regr.predict(features_test)) ** 0.5)

RMSE случайной модели: 4571.63251576971
CPU times: user 3 ms, sys: 36 µs, total: 3.04 ms
Wall time: 3.05 ms


RMSE случайной модели получилось выше, значит, проверка на адекватность пройдена.

## Вывод

- Самой качественной моделью получился Случайный лес с параметрами `max_depth = 20` и `n_estimators = 50`. Значение RMSE на тестовой выборке получилось 1571.65. Но по времени обучения модель занимает 37 секунд, что по сравнению с CatBoost и Деревом решений достаточно долго, поэтому её не рекомендуется выбирать.
- Самой быстрой моделью и по обучению, и по предсказанию из всех получилось Дерево решений, но качество модели низкое. Значение RMSE самое худшее из всех, если не учитывать Линейную Регрессию.
- Учитывая скорость и качество модели, необходимое заказчику, рекомендовано использовать модель Градиентного бустинга CatBoost с параметрами `depth = 9` и `learning_rate = 0.7`. Время обучения у модели 3 секунды, а предсказания 20 миллисекунд, а значение RMSE = 1645.72. 